
# レイヤーとモジュール

:label: `sec_model_construction`

私たちが初めてニューラル ネットワークを導入したとき、単一の出力を持つ線形モデルに焦点を当てました。ここでは、モデル全体が 1 つのニューロンだけで構成されています。単一のニューロン (i) が何らかの入力セットを受け取ることに注意してください。 (ii) 対応するスカラー出力を生成します。 (iii) 対象となる目的関数を最適化するために更新できる関連パラメータのセットを持っています。次に、複数の出力を持つネットワークについて考え始めると、ベクトル化された演算を利用してニューロンの層全体を特徴付けました。個々のニューロンと同様に、層は (i) 一連の入力を受け取り、(ii) 対応する出力を生成し、(iii) 一連の調整可能なパラメーターによって記述されます。ソフトマックス回帰を行ったとき、単一レイヤー自体がモデルでした。ただし、その後 MLP を導入したときでも、このモデルは同じ基本構造を保持していると考えることができます。

興味深いことに、MLP の場合、モデル全体とその構成層の両方がこの構造を共有しています。モデル全体は、生の入力 (特徴) を受け取り、出力 (予測) を生成し、パラメーター (すべての構成レイヤーから結合されたパラメーター) を所有します。同様に、個々の層は入力 (前の層によって供給される) を取り込み、出力 (後続の層への入力) を生成し、後続の層から逆流する信号に従って更新される一連の調整可能なパラメーターを保持します。

ニューロン、レイヤー、モデルはビジネスを進めるのに十分な抽象化を提供すると思うかもしれませんが、実際には、個々のレイヤーよりも大きく、モデル全体よりは小さいコンポーネントについて話すと便利であることがよくわかります。たとえば、コンピューター ビジョンで広く普及している ResNet-152 アーキテクチャには、数百のレイヤーがあります。これらのレイヤーは、*レイヤーのグループ*の繰り返しパターンで構成されます。このようなネットワークを一度に 1 層ずつ実装すると、退屈になる可能性があります。この懸念は単なる仮説ではありません。このような設計パターンは実際には一般的です。前述の ResNet アーキテクチャは、2015 年の ImageNet および COCO コンピュータ ビジョン コンペティションで認識と検出の両方で優勝しており (引用: `He.Zhang.Ren.ea.2016` )、今でも多くのビジョン タスクで頼りになるアーキテクチャです。レイヤーがさまざまな繰り返しパターンで配置される同様のアーキテクチャは、現在、自然言語処理や音声などの他の分野で広く普及しています。

これらの複雑なネットワークを実装するために、ニューラル ネットワーク*モジュール*の概念を導入します。モジュールは、単一のレイヤー、複数のレイヤーで構成されるコンポーネント、またはモデル全体を記述することができます。モジュール抽象化を使用する利点の 1 つは、それらをより大きな成果物に (多くの場合再帰的に) 結合できることです。これは :numref: `fig_blocks`に示されています。オンデマンドで任意の複雑さのモジュールを生成するコードを定義することで、驚くほどコンパクトなコードを記述しながら、複雑なニューラル ネットワークを実装できます。 

![](http://d2l.ai/_images/blocks.svg) :label: `fig_blocks`

プログラミングの観点から見ると、モジュールは*クラス*によって表されます。そのサブクラスは、その入力を出力に変換する順伝播メソッドを定義し、必要なパラメーターを格納する必要があります。一部のモジュールはパラメータをまったく必要としないことに注意してください。最後に、モジュールは勾配を計算するために逆伝播法を備えている必要があります。幸いなことに、独自のモジュールを定義するときに自動微分 ( :numref: `sec_autograd`で導入) によって提供される舞台裏の魔法のおかげで、パラメーターと順伝播メソッドについてのみ考慮する必要があります。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F


[**まず、MLP の実装に使用したコードを再確認します**] (:numref: `sec_mlp` )。次のコードは、256 ユニットと ReLU アクティベーションを備えた 1 つの完全接続された隠れ層を備えたネットワークを生成し、その後に 10 ユニットを備えた完全接続された出力層 (アクティベーション関数なし) を生成します。


In [2]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])


この例では、引数として渡される実行順序のレイヤーを使用して、 `nn.Sequential`をインスタンス化することによってモデルを構築しました。つまり、( **`nn.Sequential`特別な種類の`Module`を定義します**)、PyTorch でモジュールを表すクラスです。これは、構成要素である`Module`の順序付きリストを維持します。完全に接続された 2 つの層はそれぞれ、それ自体が`Module`のサブクラスである`Linear`クラスのインスタンスであることに注意してください。順伝播 ( `forward` ) メソッドも非常に単純です。リスト内の各モジュールを連結し、それぞれの出力を入力として次のモジュールに渡します。これまで、コンストラクション`net(X)`を介してモデルを呼び出して出力を取得していることに注意してください。これは実際には`net.__call__(X)`の単なる省略表現です。



## [**カスタムモジュール**]

おそらく、モジュールがどのように機能するかについての直感を養う最も簡単な方法は、モジュールを自分で実装することです。独自のカスタム モジュールを実装する前に、各モジュールが提供する必要がある基本機能を簡単にまとめます。
1. 順伝播メソッドへの引数として入力データを取り込みます。
1. 順伝播メソッドに値を返させることで出力を生成します。出力の形状は入力とは異なる場合があることに注意してください。たとえば、上記のモデルの最初の完全に接続された層は、任意の次元の入力を取り込みますが、次元 256 の出力を返します。
1. 入力に対する出力の勾配を計算します。これは逆伝播法を介してアクセスできます。通常、これは自動的に行われます。
1. 順伝播計算を実行するために必要なパラメータを保存し、それらのパラメータへのアクセスを提供します。
1. 必要に応じてモデルパラメータを初期化します。

次のスニペットでは、256 個の隠れユニットを持つ 1 つの隠れ層と 10 次元の出力層を備えた MLP に対応するモジュールを最初からコーディングします。以下の`MLP`クラスはモジュールを表すクラスを継承していることに注意してください。親クラスのメソッドに大きく依存し、独自のコンストラクター (Python の`__init__`メソッド) と順伝播メソッドのみを提供します。


In [3]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


まずは順伝播法に注目してみましょう。 `X`を入力として受け取り、活性化関数を適用して隠れた表現を計算し、そのロジットを出力することに注意してください。この`MLP`実装では、両方の層がインスタンス変数です。これが合理的である理由を理解するには、2 つの MLP `net1`と`net2`インスタンス化し、それらを異なるデータでトレーニングすることを想像してください。当然のことながら、それらが 2 つの異なる学習済みモデルを表すことが期待されます。

順伝播メソッドを**呼び出すたびに、コンストラクターで [MLP のレイヤーをインスタンス化**] (**その後、これらのレイヤーを呼び出します**) を行います。いくつかの重要な詳細に注意してください。まず、カスタマイズされた`__init__`メソッドは`super().__init__()`を介して親クラスの`__init__`メソッドを呼び出すため、ほとんどのモジュールに適用できる定型コードを再記述する手間が省けます。次に、完全に接続された 2 つのレイヤーをインスタンス化し、それらを`self.hidden`と`self.out`に割り当てます。新しい層を実装しない限り、逆伝播法やパラメータの初期化について心配する必要はないことに注意してください。システムはこれらのメソッドを自動的に生成します。これを試してみましょう。


In [4]:
net = MLP()
net(X).shape

torch.Size([2, 10])


モジュール抽象化の重要な利点は、その多用途性です。モジュールをサブクラス化して、層 (完全に接続された層クラスなど)、モデル全体 (上記の`MLP`クラスなど)、または中程度の複雑さのさまざまなコンポーネントを作成できます。畳み込みニューラル ネットワークに対処する場合など、次の章全体でこの多用途性を活用します。

##  [**シーケンシャルモジュール**]

 :label: `subsec_model-construction-sequential`

ここで、 `Sequential`クラスがどのように機能するかを詳しく見てみましょう。 `Sequential`他のモジュールをデイジーチェーン接続するように設計されていることを思い出してください。独自の単純化された`MySequential`を構築するには、2 つの主要なメソッドを定義するだけです。
1. モジュールを 1 つずつリストに追加するメソッド。
1. モジュールのチェーンを介して、追加されたのと同じ順序で入力を渡す順伝播メソッド。

次の`MySequential`クラスは、デフォルトの`Sequential`クラスと同じ機能を提供します。


In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X


`__init__`メソッドでは、 `add_modules`メソッドを呼び出してすべてのモジュールを追加します。これらのモジュールには、後で`children`メソッドからアクセスできます。このようにして、システムは追加されたモジュールを認識し、各モジュールのパラメータを適切に初期化します。



`MySequential`の順伝播メソッドが呼び出されると、追加された各モジュールが追加された順序で実行されます。 `MySequential`クラスを使用して MLP を再実装できるようになりました。


In [6]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])


この`MySequential`の使用法は、( :numref: `sec_mlp`で説明したように) `Sequential`クラス用に以前に作成したコードと同じであることに注意してください。

##  [**順伝播法でのコードの実行**]

 `Sequential`クラスを使用するとモデルの構築が簡単になり、独自のクラスを定義せずに新しいアーキテクチャを組み立てることができます。ただし、すべてのアーキテクチャが単純なデイジーチェーンであるわけではありません。より高い柔軟性が必要な場合は、独自のブロックを定義する必要があります。たとえば、順伝播メソッド内で Python の制御フローを実行したい場合があります。さらに、単に事前定義されたニューラル ネットワーク層に依存するのではなく、任意の数学的演算を実行したい場合もあります。

お気づきかと思いますが、これまでネットワーク内のすべての操作は、ネットワークのアクティベーションとそのパラメーターに基づいて行われていました。ただし、場合によっては、前のレイヤーの結果でも更新可能なパラメーターでもない項を組み込みたい場合があります。これらを*定数パラメータ*と呼びます。たとえば、関数 $f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$ を計算する層が必要だとします。$\mathbf {x}$ は入力、$\mathbf{w}$ はパラメータ、$c$ は最適化中に更新されない指定された定数です。そこで、次のように`FixedHiddenMLP`クラスを実装します。


In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


この`FixedHiddenMLP`モデルでは、隠れ層を実装します。その重み( `self.rand_weight` )はインスタンス化時にランダムに初期化され、その後は一定になります。この重みはモデル パラメーターではないため、バックプロパゲーションによって更新されることはありません。次に、ネットワークはこの「固定」層の出力を完全に接続された層に渡します。

出力を返す前に、モデルが何か異常なことを行ったことに注意してください。 while ループを実行し、$\ell_1$ ノルムが $1$ より大きいという条件でテストし、条件を満たすまで出力ベクトルを $2$ で除算しました。最後に、 `X`のエントリの合計を返しました。私たちの知る限り、この操作を実行する標準的なニューラル ネットワークはありません。この特定の操作は、実際のタスクでは役に立たない可能性があることに注意してください。私たちの目的は、任意のコードをニューラル ネットワークの計算フローに統合する方法を示すことだけです。


In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1058, grad_fn=<SumBackward0>)


[**さまざまな方法でモジュールを組み立てることができます。** ] 次の例では、創造的な方法でモジュールをネストします。


In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.0964, grad_fn=<SumBackward0>)


## まとめ

レイヤーはモジュールです。多くの層がモジュールを構成できます。多くのモジュールが 1 つのモジュールを構成することができます。

モジュールにはコードを含めることができます。モジュールは、パラメータの初期化や逆伝播などの多くのハウスキーピングを処理します。レイヤーとモジュールの順次連結は、 `Sequential`モジュールによって処理されます。

## 演習
1. モジュールを Python リストに格納するように`MySequential`を変更すると、どのような問題が発生しますか?
1.  `net1`と`net2`の 2 つのモジュールを引数として受け取り、順伝播で両方のネットワークの連結された出力を返すモジュールを実装します。これは並列モジュールとも呼ばれます。
1. 同じネットワークの複数のインスタンスを連結するとします。同じモジュールの複数のインスタンスを生成するファクトリ関数を実装し、そこから大規模なネットワークを構築します。



[ディスカッション](https://discuss.d2l.ai/t/55)
